<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/P_SAT_%EB%94%A5%EB%9F%AC%EB%8B%9D%ED%8C%80_%EC%A3%BC%EC%A0%9C%EB%B6%84%EC%84%9D_1%EC%A3%BC%EC%B0%A8_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

딥러닝팀 주제분석으로 화자 대화를 기반으로 한 이모티콘 추천 시스템 및 인기 이모티콘 추천 시스템을 마련해보고자 한다. 이를 위해 우선 기존에 존재하는 인기 이모티콘을 크롤링 할 수 있는 코드를 작성했다. 이를 통해 인기 이모티콘 순위와 각 이모티콘의 이미지를 데이터화 할 수 있게 되었다. 뿌듯!

##1. 셀레늄 설치 (코랩으로 실행)


In [ ]:
!pip install Selenium

!apt-get update # to update ubuntu to correctly run apt install

!apt install chromium-chromedriver

## 2. 라이브러리 불러오기 


In [4]:
from selenium import webdriver
import time
import os
import numpy as np

## 3. 테스트 실행

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

wd.get("http://naver.com")
print(wd.page_source)

## 3. 카카오 이모티콘샵 접근 및 이모티콘 이름과 랭킹 추출

In [8]:
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
wd.get("https://e.kakao.com/item/hot")

# 이모티콘 랭킹 20*n개를 뽑기 위해 더보기 칸 누르기 
for i in range(3):
    scroll = wd.find_element_by_css_selector("#mArticle > div:nth-child(3) > div > div.style__EmoticonMore-bg8i7m-0.flwQNQ") # 한번만 찾으면 될 줄 알았는데 매 스크롤마다 새로 찾아야했다. 
    scroll.click()
    time.sleep(2) # 로딩 시간 고려해서 2초 쉬기 

emo_name_list = wd.find_elements_by_class_name('style__TitEmoticon-mzhzui-6.cejKrV') # 이모티콘 이름 값이 담긴 리스트
emo_link_list = wd.find_elements_by_class_name('style__Li-mzhzui-1.eWKIEK') # 이모티콘 링크 값이 담긴 리스트

emogi_name = [name.text for name in emo_name_list] # 이모티콘 이름 리스트
emogi_rank = range(1, (len(emogi_name)+1)) # 이모티콘 랭크도 리스트로 만들기

emogi_link = list()
for num, link in enumerate(emo_link_list):
    emo_find = link.find_element_by_tag_name('li > a')
    emogi_link.append(emo_find.get_attribute('href')) # 링크값 뽑기 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


## 4. 각각의 이모티콘 이미지 저장하기 

In [19]:
# 실행 전 구글 드라이브 마운트하기
dir_tmp = "/content/drive/My Drive/P-SAT/2020-2/주제분석/크롤링/"
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

for link, name in zip(emogi_link, emogi_rank):
    dir = dir_tmp + str(name)
    os.makedirs(dir)
    
    wd.get(link)
    img_list = wd.find_elements_by_class_name('style__LinkSend-s2w8d1-2.hokluw')
    for num, cap in enumerate(img_list):
        img = cap.screenshot_as_png
        with open(f"{dir}/{str(name)}_{num}.png", "wb") as file:
            file.write(img)
    print(f"{emogi_rank}번째 이모티콘 모두 캡쳐 완료")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


한가지 아쉬운 건 이모티콘 이름들 중에 기호가 들어간 경우에 디렉토리 만드는 것부터 오류가 발생했다. 그래서 랭킹으로 이름을 대체해서 저장했는데, 나중에 데이터를 사용할때, 이름과 랭킹을 딕셔너리로 바꿔서 사용해야 할 것 같다. 